In [1]:
import os
import json

import random
import numpy as np
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset, RandomSampler

from torch.nn import CrossEntropyLoss, MSELoss
from sklearn.metrics import recall_score, precision_score, f1_score

In [2]:
def get_dataset(file_name):
    f = open(file_name, 'r')
    data = json.load(fp = f)
    f.close()

    dataset = {}
    for x, y in data:
        if ((x not in dataset) and (y != -1)):
            dataset[x] = y
    return dataset

posi_data = get_dataset('../data/deep-learning.jsonl')
nega_data = get_dataset('../data/android.jsonl')

In [3]:
posi_data = list(posi_data.values())
nega_data = list(nega_data.values())
dataset = [[x, 1] for x in posi_data] + [[x, 0] for x in nega_data]
random.shuffle(dataset)

In [4]:
class arguments(object):
    def __init__(self):
        pass
args = arguments()

In [5]:
args.epochs = 100
args.batch_size = 4
args.learning_rate = 1e-5
args.dropout_rate = 0.1

args.seed = 978438233
args.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
args.n_gpu = torch.cuda.device_count()

In [6]:
def set_seed():
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)
    if args.n_gpu > 0:
        torch.cuda.manual_seed_all(args.seed)
set_seed()

In [7]:
class TextDataset(Dataset):
    def __init__(self, examples):
        self.examples = examples
        
    def __len__(self):
        return len(self.examples)
        
    def __getitem__(self, item):
        x, y = self.examples[item]
        return torch.tensor(x), torch.Tensor([y])


train_data = TextDataset(dataset[: int(len(dataset) * 0.75)])
train_sampler = RandomSampler(train_data)
train_dataloader = DataLoader(train_data, sampler = train_sampler, drop_last = True,
                              num_workers = 4, batch_size = args.batch_size)

test_data = TextDataset(dataset[int(len(dataset) * 0.75) :])
test_sampler = RandomSampler(test_data)
test_dataloader = DataLoader(test_data, sampler = test_sampler, drop_last = False,
                             num_workers = 4, batch_size = args.batch_size)

In [8]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.fc1 = nn.Linear(768, 768 * 4)
        self.fc2 = nn.Linear(768 * 4, 768)
        self.fc3 = nn.Linear(768, 2)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(args.dropout_rate)
        
    def forward(self, x, labels = None):
        x = self.dropout(x)
        x = F.relu(self.fc1(x))
        x = self.dropout(x)
        x = F.relu(self.fc2(x))
        x = self.dropout(x)
        x = self.fc3(x)
        
        x = F.softmax(x, dim = 1)[:, 1]
        if (labels is None):
            return x
        else:
            loss_function = MSELoss()
            loss = loss_function(x.view(-1), labels.view(-1))
            return loss

In [9]:
def evaluate(model, epoch_id):
    y_trues = []
    y_preds = []
    bar = tqdm(test_dataloader, total = len(test_dataloader))
    for data in bar:
        x, labels = data
        x = x.to(args.device)
        labels = labels.to(args.device)
        model.eval()
        with torch.no_grad():
            prob = model(x).view(-1)
            y_preds.append((prob > 0.5).long().cpu().numpy())
            y_trues.append(labels.long().view(-1).cpu().numpy())
    y_trues = np.concatenate(y_trues, 0)
    y_preds = np.concatenate(y_preds, 0)
    TP = sum([x == 1 and y == 1 for x, y in zip(y_trues, y_preds)])
    FP = sum([x == 0 and y == 1 for x, y in zip(y_trues, y_preds)])
    TN = sum([x == 0 and y == 0 for x, y in zip(y_trues, y_preds)])
    FN = sum([x == 1 and y == 0 for x, y in zip(y_trues, y_preds)])
    print('TP FP TN FN =', TP, FP, TN, FN)

    f1 = float(f1_score(y_trues, y_preds))
    rs = float(recall_score(y_trues, y_preds))
    ps = float(precision_score(y_trues, y_preds))
    print('f1:', f1)
    print('recall:', rs)
    print('precision:', ps)
#     os.system('mkdir -p result')
#     f = open('result/baseline' + str(args.current_topic).zfill(2) + '-' + str(epoch_id).zfill(3) + '.txt', 'w')
#     print(f1, file = f)
#     f.close()
#     return f1

model = Model()
model.to(args.device)
if args.n_gpu > 1:
    model = torch.nn.DataParallel(model)
optimizer = torch.optim.Adam(model.parameters(), lr = args.learning_rate)

for epoch_num in range(args.epochs):
    bar = tqdm(train_dataloader, total = len(train_dataloader))
    for data in bar:
        x, labels = data
        x = x.to(args.device)
        labels = labels.to(args.device)
        model.train()
        loss = model(x, labels)
        if args.n_gpu > 1:
            loss = loss.mean()
        loss.backward()
        bar.set_description("epoch {}".format(epoch_num))
        optimizer.step()
        optimizer.zero_grad()
    evaluate(model, epoch_num)

100%|████████████████████████████████████████| 277/277 [00:00<00:00, 745.89it/s]


TP FP TN FN = 551 246 271 38
f1: 0.795093795093795
recall: 0.9354838709677419
precision: 0.6913425345043914


100%|████████████████████████████████████████| 277/277 [00:00<00:00, 755.97it/s]


TP FP TN FN = 525 42 475 64
f1: 0.9083044982698962
recall: 0.8913412563667232
precision: 0.9259259259259259


100%|████████████████████████████████████████| 277/277 [00:00<00:00, 774.34it/s]


...

TP FP TN FN = 554 17 500 35
f1: 0.9551724137931035
recall: 0.9405772495755518
precision: 0.9702276707530648


100%|████████████████████████████████████████| 277/277 [00:00<00:00, 758.63it/s]


TP FP TN FN = 551 14 503 38
f1: 0.9549393414211438
recall: 0.9354838709677419
precision: 0.9752212389380531


100%|████████████████████████████████████████| 277/277 [00:00<00:00, 769.85it/s]

TP FP TN FN = 551 14 503 38
f1: 0.9549393414211438
recall: 0.9354838709677419
precision: 0.9752212389380531
